In [1]:
using Statistics
using Distributions
using Random
using Combinatorics
import LinearAlgebra: dot
using CSV
using DataFrames

In [2]:
"""
    generate_random_digraph(V::Int64, p::Float64)

Generate a random directed graph with `V` vertices.
Draws from a uniform distribution over directed graphs;
every edge has probability `p` of being in the graph. 
"""
function generate_random_digraph(V::Int64, p::Float64)
    parent_sets = [convert(Vector{Bool}, rand(Distributions.Bernoulli(p), V)) for i=1:V]
    return parent_sets
end


"""
    digraph_prior(lambda::Float64, reference::Vector{Vector{Bool}})

Sample a new graph G from the density

P(G) \\propto exp(-lambda * |G \\ G'|)

Where G' is a reference graph.
"""
function digraph_prior(lambda::Float64, reference::Vector{Vector{Bool}})

    result = []
    prob = exp(-lambda) / (1.0 + exp(-lambda))
    for (i, ps) in enumerate(reference)
        new_ps = Vector{Bool}()
        for (j, parent) in enumerate(ps)
            if parent
                bp = 0.5
            else
                bp = prob
            end
            push!(new_ps, rand(Distributions.Bernoulli(bp)))
        end
        push!(result, new_ps)
    end
    return result
end


"""
    modify_digraph!(ps::Vector{Vector{Int64}}, keep::Int64, add::Int64)

Modify a given graph
"""
function modify_digraph!(ps::Vector{Vector{Bool}}, keep::Int64, add::Int64)

    # get the indices of the original edges
    ps_idx = []
    not_ps_idx = [] # and the complement
    for i=1:length(ps)
        for (j, b) in enumerate(ps[i])
            if b
                push!(ps_idx, (i,j))
            else
                push!(not_ps_idx, (i,j))
            end
        end
    end
        
    # Randomly remove edges from the graph
    n_edges = length(ps_idx)
    to_remove = Random.randperm(n_edges)[1:max(n_edges - keep, 0)]
    for rm_idx in to_remove
        (i,j) = ps_idx[rm_idx]
        ps[i][j] = false
    end
    
    # Randomly add edges (from the complement)
    n_not_edges = length(not_ps_idx)
    to_add = Random.randperm(n_not_edges)[1:add]
    for add_idx in to_add
        (i,j) = not_ps_idx[add_idx]
        ps[i][j] = true
    end
    
    return ps
end


"""
    n_coeffs(V::Int64, max_degree::Int64)

Given a number of variables (V) and a maximum polynomial degree
for the regression model, compute the number of coefficients required
by the model. (this includes the constant term)
"""
function n_coeffs(V::Int64, max_degree::Int64)
    return sum([binomial(V, i) for i=0:min(V, max_degree)])
end
    


"""
    generate_reg_coeffs(ps::Vector{Vector{Bool}}, coeff_std::Float64, regression_deg::Int64)

Given a vector of parent sets, generate a vector of regression coefficient vectors.
(This includes the constant term)
"""
function generate_reg_coeffs(ps::Vector{Vector{Bool}}, coeff_std::Float64, regression_deg::Int64)
   
    result = Vector{Vector{Float64}}()
    for v in ps
        num_vars = sum(v)
        bw = n_coeffs(num_vars, regression_deg)
        push!(result, coeff_std.*randn(bw))
    end
    
    return result
end


"""
    initialize_time_series(V)
"""
function initialize_time_series(V::Int64; init_std::Float64=1.0)
    return init_std.*randn(V)
end


"""
    compute_B_data(x_vec::Vector{Float64}, ps::Vector{Bool}, regression_deg::Int64)
"""
function compute_B_data(x_vec::Vector{Float64}, ps::Vector{Bool}, regression_deg::Int64)
   
    used_x = x_vec[ps]
    num_parents = length(used_x)
    
    B_vec = ones(n_coeffs(num_parents, regression_deg))
    
    col = 2
    for deg=1:regression_deg
        for comb in Combinatorics.combinations(used_x, deg)
            B_vec[col] = prod(comb)
            col += 1
        end
    end
    
    return B_vec
end

"""
    generate_next_timestep_data(x_vec::Vector{Float64}, parent_sets::Vector{Vector{Bool}}, regression_std::Int64)
"""
function generate_next_timestep_data(x_vec::Vector{Float64}, 
                                     parent_sets::Vector{Vector{Bool}},
                                     regression_coeffs::Vector{Vector{Float64}},
                                     regression_deg::Int64,
                                     regression_std::Float64)
    
    next_x = Vector{Float64}()
    
    for (i, ps) in enumerate(parent_sets)
        
        data_vec = compute_B_data(x_vec, ps, regression_deg)
        
        push!(next_x, dot(data_vec, regression_coeffs[i]) + regression_std*randn() )
    end
    
    return next_x
end


"""
    generate_time_series(parent_sets::Vector{Vector{Bool}}, 
                         V::Int64, T::Int64)

Create a simulated time series with `T` time steps
"""
function generate_time_series(parent_sets::Vector{Vector{Bool}},
                              T::Int64,
                              regression_coeffs::Vector{Vector{Float64}},
                              regression_deg::Int64,
                              regression_std::Float64)
    
    result = zeros(T, length(parent_sets))
    result[1,:] = initialize_time_series(length(parent_sets); init_std=regression_std)
    
    for t=2:T
        result[t,:] = generate_next_timestep_data(result[t-1,:], parent_sets,
                                                  regression_coeffs,
                                                  regression_deg,
                                                  regression_std)
    end
    
    return result
end


"""
    generate_dataset(T::Int64, N::Int64, parent_sets::Vector{Vector{Bool}},
                     coeff_std::Float64, regression_deg::Int64, regression_std::Float64)

Given a graph structure, generate some coefficients and simulate a dataset of N time series,
each time series over T timesteps.
"""
function generate_dataset(T::Int64, N::Int64,
                          parent_sets::Vector{Vector{Bool}},
                          coeff_std::Float64, 
                          regression_deg::Int64, regression_std::Float64)
    
    coeffs = generate_reg_coeffs(parent_sets, coeff_std, regression_deg)
    
    return [generate_time_series(parent_sets, T, coeffs, regression_deg, regression_std) for i=1:N]
    
end



"""
    modify_and_simulate(ref_ps::Vector{Vector{Bool}}, remove::Int64, add::Int64)

Given a reference graph, create a modified version of it and then simulate a dataset
from that modified graph.
"""
function modify_and_simulate(ref_ps::Vector{Vector{Bool}}, keep::Int64, add::Int64, 
                             T::Int64, N::Int64,
                             coeff_std::Float64, 
                             regression_deg::Int64, regression_std::Float64)
    
    ref_ps_copy = [copy(ps) for ps in ref_ps]
    modify_digraph!(ref_ps_copy, keep, add)
    
    ds = generate_dataset(T, N, ref_ps_copy, coeff_std, regression_deg, regression_std) 
    
    return ref_ps_copy, ds
end

"""
    save_dataset(dataset::Vector{Array{Float64,2}}, file_name::String)
"""
function save_dataset(dataset::Vector{Array{Float64,2}}, file_name::String)

    V = size(dataset[1],2)
    
    df = DataFrames.DataFrame(Dict([:timeseries=>Int[]; :timestep=>Int[]; 
                                   [(Symbol("var",i)=>Float64[]) for i=1:V]]))    
    
    for (i, timeseries) in enumerate(dataset)
        for t=1:size(timeseries,1)
            push!(df, [i; t; dataset[i][t,:]])
        end
    end

    CSV.write(file_name, df; delim="\t")
end
    

"""
    save_graph(parent_sets::Vector{Vector{Float64}}, file_name::String)
"""
function save_graph(parent_sets::Vector{Vector{Bool}}, file_name::String)
   
    CSV.write(file_name, 
              DataFrame(convert(Matrix{Int64}, hcat(parent_sets...)));
              delim=",", writeheader=false)
    
end

save_graph

In [3]:
ref = generate_random_digraph(20,0.1)

20-element Array{Array{Bool,1},1}:
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
 [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
 [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [4]:
convert(Matrix{Float64}, hcat(ref...))

20×20 Array{Float64,2}:
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  1.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  1.0  1.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     1.0  0.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  …  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [6]:
CSV.write("dumb_reference.csv", DataFrame(convert(Matrix{Int64}, hcat(ref...))); delim=",", writeheader=false)

"dumb_reference.csv"

In [7]:
keep = 20
add = 10
T = 100
N = 3
coeff_std = 1.0/length(ref)
regression_std = 1.0/sqrt(length(ref))
regression_deg = 4
regression_std = 1.0
mod_ref, ds = modify_and_simulate(ref, keep, add, T, N, coeff_std, regression_deg, regression_std)

(Array{Bool,1}[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [8]:
save_dataset(ds, "dumb_time_series.tsv")
save_graph(mod_ref, "dumb_true.csv")

"dumb_true.csv"

In [9]:
df = CSV.read("dumb_time_series.tsv", delim="\t")

,timeseries,timestep,var1,var10,var11,var12,var13
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,0.0228698,-0.113172,-0.554446,-0.33393,-0.874112
2,1.0,2.0,-0.0114158,0.9312,1.54879,-1.30569,-0.501856
3,1.0,3.0,0.917438,-0.94588,-0.364811,-0.297199,-0.682692
4,1.0,4.0,-0.973552,-0.889696,0.154459,-1.18881,0.309108
5,1.0,5.0,1.3988,-0.57887,1.21847,0.203799,-0.498574
6,1.0,6.0,-1.27066,1.23356,0.882955,-0.768622,-1.23042
7,1.0,7.0,-0.34017,-0.466005,0.0596527,-0.184856,1.771
8,1.0,8.0,1.26101,-1.15724,1.62964,0.547141,0.35657
9,1.0,9.0,0.76825,1.25536,1.67268,-0.473008,1.39981


In [ ]:
function standard_timeseries_preprocess!(timeseries_df)

    result = Vector{Matrix{Float64}}()

    N = convert(Int64, maximum(timeseries_df.timeseries))

    for i=1:N
        ts = timeseries_df[timeseries_df.timeseries .== i, :]
        sort!(ts, [:timestep])
        push!(result, convert(Matrix{Float64}, ts[:,3:end]))
    end

    return result
end

In [15]:
ts = df[df.timeseries .== 1,:]

,timeseries,timestep,var1,var10,var11,var12,var13
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,0.0228698,-0.113172,-0.554446,-0.33393,-0.874112
2,1.0,2.0,-0.0114158,0.9312,1.54879,-1.30569,-0.501856
3,1.0,3.0,0.917438,-0.94588,-0.364811,-0.297199,-0.682692
4,1.0,4.0,-0.973552,-0.889696,0.154459,-1.18881,0.309108
5,1.0,5.0,1.3988,-0.57887,1.21847,0.203799,-0.498574
6,1.0,6.0,-1.27066,1.23356,0.882955,-0.768622,-1.23042
7,1.0,7.0,-0.34017,-0.466005,0.0596527,-0.184856,1.771
8,1.0,8.0,1.26101,-1.15724,1.62964,0.547141,0.35657
9,1.0,9.0,0.76825,1.25536,1.67268,-0.473008,1.39981


In [18]:
sort(ts, [:timestep], rev=true)

,timeseries,timestep,var1,var10,var11,var12,var13
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,100.0,-0.421608,0.400224,-0.0482371,1.5605,-0.799362
2,1.0,99.0,0.81923,-2.34334,0.746829,0.443526,-0.702731
3,1.0,98.0,-1.56507,-1.13193,-0.265053,-0.0839415,1.45153
4,1.0,97.0,0.997444,0.267112,-0.227374,0.651756,-0.435146
5,1.0,96.0,-0.551784,0.559848,-1.01103,-0.509251,1.35121
6,1.0,95.0,0.446055,0.614887,0.946437,0.801655,-0.416037
7,1.0,94.0,-0.161474,0.475504,0.0908018,-0.254955,-0.232692
8,1.0,93.0,0.881706,1.33298,-0.875379,-0.551624,-1.8928
9,1.0,92.0,1.34774,0.431132,-0.870173,-0.560929,0.875882


In [19]:
ts[:,3:end]

,var1,var10,var11,var12,var13,var14,var15
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0228698,-0.113172,-0.554446,-0.33393,-0.874112,-0.168852,0.384099
2,-0.0114158,0.9312,1.54879,-1.30569,-0.501856,-1.25952,-3.291
3,0.917438,-0.94588,-0.364811,-0.297199,-0.682692,0.746672,0.519302
4,-0.973552,-0.889696,0.154459,-1.18881,0.309108,-0.739151,1.34489
5,1.3988,-0.57887,1.21847,0.203799,-0.498574,-0.528715,-2.69129
6,-1.27066,1.23356,0.882955,-0.768622,-1.23042,-0.449887,-0.294939
7,-0.34017,-0.466005,0.0596527,-0.184856,1.771,1.1778,-1.24698
8,1.26101,-1.15724,1.62964,0.547141,0.35657,-0.794436,1.93525
9,0.76825,1.25536,1.67268,-0.473008,1.39981,-1.25344,-0.435413


In [20]:
ps_df = CSV.read("dumb_reference.csv")

,0,1,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8,0_9,0_10
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,0,0,0,1,1,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,1,1,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,1,0,0,0
6,0,0,0,0,1,0,0,0,0,0,0,0
7,0,0,0,1,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,1,0
9,0,0,0,0,0,0,0,1,1,0,0,0


In [21]:
convert(Matrix{Bool}, ps_df)

19×20 Array{Bool,2}:
 0  0  0  0  1  1  0  0  0  0  0  0  0  1  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  1  1  0  0
 0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  1  1  1  0
 0  0  0  0  0  0  0  1  1  0  0  0  0  1  0  0  1  0  0  0
 1  0  1  0  0  0  0  0  1  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  1
 0  1  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  1  1  0  0  0  1  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1  0  0  0
 0  1  1  0  0  0  0  0  0  0  0  0  0  1  0  1  1  0  0  0
 0  0  0  0  0  1  

In [ ]:
pg = digraph_prior(3.0, ref)

In [ ]:
pgc = [copy(pg_v) for pg_v in pg]

In [ ]:
modify_digraph!(pgc, 1, 2)

In [ ]:
convert(Vector{Bool}, rand(Distributions.Bernoulli(0.25), 20))

In [ ]:
using Distributions
rand(Distributions.Bernoulli(0.25))

In [ ]:
rand(Distributions.Bernoulli(0.25))

In [ ]:
ls = [1,2,3,4]

In [ ]:
ls[1:0]

In [ ]:
randn(10)

In [ ]:
using Combinatorics

In [ ]:
for comb in Combinatorics.combinations([1,2,3,4], 2)
    println( prod(comb))
end

In [ ]:
compute_B_data([0.1, 0.2, 0.3, 0.4, 0.5, 0.6], [true, true, true, true, false, false], 4)

In [ ]:
import LinearAlgebra: dot 

dot([1,2,3],[1,2,3])

In [ ]:
Dict{Symbol,Vector{Union{Int64,Float64}}}([:timeseries=>Int[]; :timestep=>Int[]])#; [ (Symbol("var",i)=>Float64[]) for i=1:3]])

In [ ]:
df = DataFrames.DataFrame(Dict([:timeseries=>Int[]; :timestep=>Int[]; [ (Symbol("var",i)=>Float64[]) for i=1:3]]))

In [ ]:
Symbol("little", 123)